# IMPORTING AND FILTERING THE NEWS

In [1]:
# !pip install -U spacy
# !pip install -U spacy-transformers
#!python -m spacy download en_core_web_trf

In [1]:
import pandas as pd
import spacy
import pickle
import re

# Load the transformer model
nlp = spacy.load("en_core_web_trf")


file_path = '/Users/aakarshsurendra/Desktop/ROAD_NLP/02_dataset/data_road.csv'

dataframe = pd.read_csv(file_path)


/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
!python -m spacy validate

/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation:
/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔
en_core_web_lg    >=3.7.2,<3.8.0   3.7.1   ✔
en_core_web_trf   >=3.7.2,<3.8.0   3.7.3   ✔



In [4]:
dataframe = dataframe.head(50) # debug

### CREATING IDs FOR EACH NEWS

In [2]:
# Function to extract id from link
def extract_id(link):
    match = re.search(r'\b(\d{6,11})\b', link)
    if match:
        return match.group(1)
    else:
        return None
    
# Apply the function to extract ids
dataframe['id'] = dataframe['Link'].apply(extract_id)
order = ['id','Link','content','News_date']
dataframe = dataframe[order]

In [3]:
# Display the DataFrame
dataframe.to_csv('dataframe_id.csv',index = False)

## TEXT PRE-PROCESSING FUNCTION

In [4]:
# Function to preprocess text using SpaCy
def preprocess_text_spacy(text):
    # Tokenize text and remove stopwords, punctuation, and lemmatize tokens
    doc = nlp(text)
    processed_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return processed_tokens

# STAGE I CLUSTERING

## DO NOT RUN 

#### Run only once and store the output TF-IDF vector file as pickle - 

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Complete data (list of article texts)
article_texts = dataframe.content

# Handling NaN values by replacing them with empty strings
article_texts_cleaned = article_texts.fillna('')

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocess_text_spacy)

# Fit and transform the data
tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe)

#### Exporting the pickle file

# Specify the file path where you want to save the TF-IDF matrix
file_path = "tfidf_matrix.pkl"

# Save the TF-IDF matrix to a file
with open(file_path, 'wb') as f:
    pickle.dump(tfidf_matrix, f)

/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Load Pickle

#### Importing the pickle file - Run this

In [4]:
# Specify the file path where the TF-IDF matrix is saved
file_path = "/Users/aakarshsurendra/Desktop/ROAD_NLP/07_pickle_files/tfidf_matrix.pkl"

# Load the TF-IDF matrix from the file
with open(file_path, 'rb') as f:
    tfidf_matrix = pickle.load(f)

In [5]:
tfidf_matrix

<50x2083 sparse matrix of type '<class 'numpy.float64'>'
	with 4581 stored elements in Compressed Sparse Row format>

### Clustering - To remove the unrelated news

In [ ]:
#Model 1
from sklearn.cluster import KMeans

# Initialize K-means clustering
num_clusters = 2  # You can adjust this based on the number of desired clusters
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=18, random_state=42)

# Fit the clustering model
kmeans.fit(tfidf_matrix)

# Get the cluster labels
cluster_labels = kmeans.labels_

### PCA

In [ ]:
import numpy as np

# Get cluster centers from K-means model
cluster_centers = kmeans.cluster_centers_

# Visualize clusters with cluster centers
plt.figure(figsize=(10, 8))
for cluster_label in range(num_clusters):
    plt.scatter(tfidf_matrix_2d[cluster_labels == cluster_label, 0],
                tfidf_matrix_2d[cluster_labels == cluster_label, 1],
                label=f'Cluster {cluster_label}')
plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1],
            marker='x', color='black', label='Cluster Centers')
plt.title('Clustering of News Articles on Road Accidents with Cluster Centers')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()


### Cluster Extraction, Model Difference and Extracting the Unwanted News Dataset

In [6]:
Model1_cluster_0_points = [i for i in range(len(tfidf_matrix_2d)) if cluster_labels[i] == 0]
Model1_cluster_1_points = [i for i in range(len(tfidf_matrix_2d)) if cluster_labels[i] == 1]


#print("Cluster 0 points:", Model1_cluster_0_points)
#print("Cluster 1 points:", Model1_cluster_1_points)

NameError: name 'tfidf_matrix_2d' is not defined

In [7]:
import random
random_10_elements = random.sample(Model1_cluster_0_points, k=15)
print(random_10_elements)

NameError: name 'Model1_cluster_0_points' is not defined

#### Verifying the unwanted news

In [15]:
import random

for index in random_10_elements:
    news_article = dataframe.loc[index, 'content']
    print(f"News Article {index}:\n{news_article}\n")

News Article 9750:
MADHYAMGRAM: A pleasure trip to Himachal Pradesh’s popular Shimla-Kullu-Manali circuit during Puja ended in tragedy for 10 tourists in Kolkata who met with an accident at Manali on Sunday, four days after they had started the journey together from Madhyamgram’s Nandankanan. The relatives of the 10 tourists from Madhyamgram, among whom a 28-year-old youth died and nine others suffered grievous injuries after their SUV plunged into a 100-metre gorge near Chaudah-Mor between Manali and Rohtang pass on Sunday morning, on Monday caught a flight and boarded a train to bring them back. Family members at the victim’s home in Chandpara The 10-member tourist team — couples Himanish Das(25) and Shampa Das (22) from Madhyamgram and Suman Das (31) and Tuktuki Das (23) from Barasat, and Subho Roy (25), Mithun Biswas (27), Salim Ali (24), Bubai Das (24), Prasanta Das (26) and Biswajit Das (28) from Madhyamgram, Sodepur and Gaighata — had started out on October 16. Gaighata resident

#### EXTRACT THE STAGE I FILTERNED NEWS

In [60]:
#Filtered Dataset from Stage I Clustering
filtered_dataset_1 = dataframe.drop(Model1_cluster_0_points)

# Reset the index of the new DataFrame
filtered_dataset_1.reset_index(drop=True, inplace=True)

In [61]:
filtered_dataset_1.to_csv('Filtered_News_1.csv', index = False)

#### EXTRACT THE UNWANTED NEWS TO FEED TO STAGE II CLUSTERING

In [17]:
unwanted_news_stage1 = dataframe.drop(Model1_cluster_1_points)
unwanted_news_stage_1.to_csv('Unwanted_News.csv', index = False)

# STAGE II CLUSTERING

## DO NOT RUN

#### Vectorize the Unwanted News

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Complete data (list of article texts)
article_texts = unwanted_news.content

# Handling NaN values by replacing them with empty strings
unwanted_article_texts_cleaned = article_texts.fillna('')

# Initialize TF-IDF vectorizer
unwanted_tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocess_text_spacy)

# Fit and transform the data
unwanted_tfidf_matrix = unwanted_tfidf_vectorizer.fit_transform(unwanted_article_texts_cleaned)

/Users/aakarshsurendra/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#### Exporting the pickle file - DO NOT RUN THIS

In [22]:
import pickle

# Specify the file path where you want to save the TF-IDF matrix
file_path = "unwanted_tfidf_matrix.pkl"

# Save the TF-IDF matrix to a file
with open(file_path, 'wb') as f:
    pickle.dump(unwanted_tfidf_matrix, f)

## Importing the pickle file - RUN THIS

In [5]:
import pickle

unwanted_news = pd.read_csv('unwanted_News.csv')
# Specify the file path where the TF-IDF matrix is saved
file_path = "unwanted_tfidf_matrix.pkl"

# Load the TF-IDF matrix from the file
with open(file_path, 'rb') as f:
    unwanted_tfidf_matrix = pickle.load(f)

#### Clustering the unwanted news for clusters

In [ ]:
from sklearn.cluster import KMeans

# Initialize K-means clustering
num_clusters = 2  # You can adjust this based on the number of desired clusters
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=15, random_state=42)

# Fit the clustering model
kmeans.fit(unwanted_tfidf_matrix)

# Get the cluster labels
cluster_labels = kmeans.labels_

In [7]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Reduce dimensionality to 2 dimensions using PCA
pca = PCA(n_components=2)
tfidf_matrix_2d = pca.fit_transform(unwanted_tfidf_matrix.toarray())


In [8]:
import numpy as np
import matplotlib.pyplot as plt

# Initialize lists to store indexes for each cluster
unwanted_cluster_0_points = []
unwanted_cluster_1_points = []

unwanted_cluster_0_points = [index for index in np.where(cluster_labels == 0)[0] if index in unwanted_news.index]
unwanted_cluster_1_points = [index for index in np.where(cluster_labels == 1)[0] if index in unwanted_news.index]


# Get cluster centers from K-means model
cluster_centers = kmeans.cluster_centers_

# Visualize clusters with cluster centers
plt.figure(figsize=(10, 8))
for cluster_label in range(num_clusters):
    cluster_indices = np.where(cluster_labels == cluster_label)[0]  # Get indices of data points in this cluster
    if cluster_label == 0:
        unwanted_cluster_0_points.extend(cluster_indices)  # Append indices to unwanted cluster 0 list
    elif cluster_label == 1:
        unwanted_cluster_1_points.extend(cluster_indices)  # Append indices to unwanted cluster 1 list
    
    plt.scatter(tfidf_matrix_2d[cluster_indices, 0],
                tfidf_matrix_2d[cluster_indices, 1],
                label=f'Cluster {cluster_label}')
    # Add labels to each point
    for i in range(len(tfidf_matrix_2d)):
        plt.text(tfidf_matrix_2d[i, 0], tfidf_matrix_2d[i, 1], str(i), fontsize=8)

plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1],
            marker='x', color='black', label='Cluster Centers')
plt.title('Clustering of News Articles on Road Accidents with Cluster Centers')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# Now you can use unwanted_cluster_0_points and unwanted_cluster_1_points to access the indexes of data points in each cluster

NameError: name 'cluster_labels' is not defined

In [83]:
import random
random_10_elements = random.sample(unwanted_cluster_1_points, k=15)
print(random_10_elements)

[3212, 1025, 342, 1707, 3208, 673, 496, 2980, 2479, 464, 1775, 1468, 900, 1516, 563]


In [84]:
import random

# Assuming random_10_elements is a list containing 10 random indexes
for index in random_10_elements:
    if index in unwanted_news.index:
        news_article = unwanted_news.loc[index, 'content']
        print(f"News Article {index}:\n{news_article}\n")

News Article 3212:
GUNTUR: In a humanitarian gesture, assembly speaker Dr Kodela Sivaprasada Rao rushed a road accident victim to the hospital by cancelling his official programmes on Tuesday night. Kodela, himself a surgeon by profession extended the first-aid at the accident spot before shifting the victim to the hospital. According to information, a shepherd Vemula Srinivasa Rao was severely injured when an unknown vehicle hit him and sheep near Issappalem village of Narasaraopet mandal. Assembly speaker Dr Kodela Sivaprasada Rao noticed Rao laying in a pool of blood while on his way to attend programmes in the mandal. The speaker immediately brought his convoy to the halt and rushed to the victim. He personally checked the pulse of the victim and shifted him into his staff car. He directed his staff to rush Rao to government hospital at Narasaraopet. Soon, Dr Kodela visited the hospital and directed the doctors to provide best medical care to shepherd Srinivasa Rao . Upon learning 

AttributeError: 'DataFrame' object has no attribute 'unwanted_cluster_1_points'

In [313]:
filtered_dataset_2 = unwanted_news.drop(unwanted_cluster_1_points)

# Reset the index of the new DataFrame
filtered_dataset_2.reset_index(drop=True, inplace=True)

KeyError: '[7, 18, 34, 38, 50, 56, 67, 113, 115, 122, 131, 132, 154, 176, 181, 204, 205, 206, 210, 214, 215, 216, 221, 222, 227, 229, 231, 251, 259, 270, 274, 279, 283, 285, 286, 297, 298, 299, 302, 303, 306, 308, 325, 329, 335, 336, 351, 355, 357, 358, 361, 364, 382, 390, 393, 408, 413, 423, 437, 442, 447, 450, 457, 461, 475, 477, 483, 489, 490, 495, 498, 501, 505, 517, 518, 519, 520, 523, 529, 530, 531, 534, 537, 543, 552, 574, 575, 580, 590, 591, 595, 607, 611, 619, 622, 623, 624, 625, 631, 647, 655, 657, 671, 672, 674, 685, 688, 692, 696, 702, 708, 726, 730, 731, 738, 741, 752, 753, 767, 782, 788, 791, 802, 817, 820, 822, 826, 830, 838, 844, 848, 849, 851, 853, 862, 863, 864, 867, 870, 877, 880, 882, 888, 908, 910, 920, 927, 931, 933, 937, 948, 951, 954, 955, 981, 985, 992, 996, 997, 1000, 1020, 1036, 1051, 1059, 1070, 1073, 1083, 1090, 1097, 1100, 1106, 1113, 1114, 1127, 1128, 1156, 1158, 1167, 1174, 1176, 1178, 1184, 1189, 1191, 1196, 1197, 1199, 1203, 1204, 1210, 1213, 1233, 123

In [257]:
filtered_dataset_2.to_csv('Filtered_News_2.csv', index = False)

NameError: name 'filtered_dataset_2' is not defined

In [119]:
filtered_dataset = pd.concat([filtered_dataset_1, filtered_dataset_2], ignore_index=True)
filtered_dataset.reset_index(drop=True, inplace=True)
filtered_dataset.to_csv('Filtered_News.csv', index = False)

# EXTRACTING FIRST SENTENCE

In [16]:
df = pd.read_csv('Filtered_News_A.csv')

In [17]:
import spacy
def One_Liners(dataset):
    nlp = spacy.load("en_core_web_trf")
    extracted_text = []

    for content in dataset['content']:
        doc = nlp(content)
        lines = content.split('\n')
        found = False
        for line in lines:
            if ":" in line:
                first_line_after_colon = line.split(":", 1)[1].strip()
                text_till_period = first_line_after_colon.split(".")[0]
                extracted_text.append(text_till_period)
                found = True
                break

        if not found:
            # Take the first row if no colon is found
            extracted_text.append(content.split(".")[0])

    # Append the extracted text to the dataset with a new column named "First Line"
    dataset['First_Line'] = extracted_text

In [35]:
One_Liners(df)

Pre-Processing

In [36]:
sentence = df.First_Line

In [37]:
df.head()

,Unnamed: 0,Link,content,News_date,First_Line
0,0,https://timesofindia.indiatimes.com/city/allah...,SULTANPUR : Three people lost their lives on F...,"Updated: Feb 23, 2024, 13:51 IST",Three people lost their lives on Friday in a c...
1,1,https://timesofindia.indiatimes.com/city/jaipu...,JAIPUR: Three people died in a car accident in...,"Updated: Feb 19, 2024, 15:48 IST",Three people died in a car accident in Rajasth...
2,2,https://timesofindia.indiatimes.com/city/benga...,RAICHUR: Two women killed and five others were...,"Updated: Feb 18, 2024, 11:34 IST",Two women killed and five others were seriousl...
3,3,https://timesofindia.indiatimes.com/city/delhi...,New Delhi: Two students were injured after a s...,"Feb 16, 2024, 08:12 IST",Two students were injured after a speeding car...
4,4,https://timesofindia.indiatimes.com/city/hyder...,"Hyderabad: A newlywed software engineer , his ...","Feb 22, 2024, 08:22 IST","A newlywed software engineer , his father-in-l..."


In [38]:
import nltk
import spacy
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

nlp = spacy.load("en_core_web_trf")

def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sentence.lower())

    # Lemmatize each word using spaCy
    lemmatized_tokens = []
    for token in nlp(sentence):
        if token.text.isalnum() and token.text not in stop_words:
            lemmatized_tokens.append(token.lemma_)

    sentence_p = " ".join(lemmatized_tokens)
    return sentence_p

df['first_line_l'] = df['First_Line'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aakarshsurendra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aakarshsurendra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [294]:
first_50_rows_first_line = df['first_line_l'].head(50)

In [295]:
# Define the output file path
output_file = 'first_line_l_text.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in first_50_rows_first_line:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")


Text file 'first_line_l_text.txt' has been created.


# ANNOTATION

## Dividing Data for Team Members Annotation

In [83]:
annotation_1000 = filtered_dataset_1.head(1000)

In [19]:
dataset = pd.read_csv('dataset_First_Line.csv')

In [20]:
annotation_1000 = dataset.iloc[1000:2000,:]

In [21]:
annotation_1000.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Link', 'content', 'News_date',
       'First_Line'],
      dtype='object')

In [66]:
One_Liners(annotation_1000)

/var/folders/p7/m480s0s56rb3pl4d11ss4wch0000gn/T/ipykernel_2234/3172095247.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['First_Line'] = extracted_text


#### Craig's File for Annotation

In [22]:
craig_annotation3 = annotation_1000['First_Line'].iloc[0:100]

In [271]:
craig_annotation2 = annotation_1000['First_Line'].iloc[600:700]

In [24]:
# Define the output file path
output_file = 'craig_annotation3.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in craig_annotation3:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")


Text file 'craig_annotation3.txt' has been created.


#### Shirin's File for Annotation

In [25]:
shirin_annotation3 = annotation_1000['First_Line'].iloc[100:200]

In [273]:
shirin_annotation2 = annotation_1000['First_Line'].iloc[700:800]

In [26]:
# Define the output file path
output_file = 'shirin_annotation3.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in shirin_annotation3:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")


Text file 'shirin_annotation3.txt' has been created.


#### Gourav's File for Annotation

In [27]:
gourav_annotation3 = annotation_1000['First_Line'].iloc[200:300]

In [236]:
gourav_annotation2 = annotation_1000['First_Line'].iloc[500:600]

In [28]:
# Define the output file path
output_file = 'gourav_annotation3.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in gourav_annotation3:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")


Text file 'gourav_annotation3.txt' has been created.


#### Aakarsh's File for Annotation

In [29]:
aakarsh_annotation3 = annotation_1000['First_Line'].iloc[300:400]

In [275]:
aakarsh_annotation2 = annotation_1000['First_Line'].iloc[800:900]

In [30]:
# Define the output file path
output_file = 'aakarsh_annotation3.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in aakarsh_annotation3:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")

Text file 'aakarsh_annotation3.txt' has been created.


#### Pallavi's File for Annotation

In [31]:
pallavi_annotation3 = annotation_1000['First_Line'].iloc[400:500]

In [279]:
pallavi_annotation2 = annotation_1000['First_Line'].iloc[900:1000]

In [32]:
# Define the output file path
output_file = 'pallavi_annotation3.txt'

# Open the output file in write mode
with open(output_file, 'w') as file:
    # Iterate over each element in the Series
    for sentence in pallavi_annotation3:
        # Write the sentence to the file
        file.write(sentence + '\n')

print(f"Text file '{output_file}' has been created.")

Text file 'pallavi_annotation3.txt' has been created.
